In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
from AutoCordon.cordon import get_road_closure_locations, split_roads_with_cordon, remove_roads_within_cordon, extract_closure_candidates
from pygeos.geometry import get_x, get_y
from pygeos.io import from_shapely
from pygeos.predicates import equals
import momepy

In [ ]:
roads = gpd.read_file(r"..\tests\data\sample_roads_soton_centre.geojson").explode().reset_index(drop=True)
road_lines = from_shapely(roads.geometry)

In [ ]:
centre = (442000, 112000)
distance = 550
distance_max = 600
closure_locations = get_road_closure_locations(centre, distance, road_lines)
split_roads = split_roads_with_cordon(centre, distance, road_lines)
new_lines = [road for road in split_roads if not equals(road, road_lines).any()]
new_lines_gdf = gpd.GeoDataFrame({"geometry": new_lines, "id": range(len(new_lines))})
remaining_roads = extract_closure_candidates(centre, distance, distance_max, road_lines)
# remaining_roads = remove_roads_within_cordon(centre, distance, road_lines)
remaining_roads_gdf = gpd.GeoDataFrame({"geometry": remaining_roads, "id": range(len(remaining_roads))}).explode().reset_index()

In [ ]:
remaining_roads_gdf

In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
roads.plot(ax=ax)
remaining_roads_gdf.plot(color="r", ax=ax)
ax.scatter(get_x(closure_locations), get_y(closure_locations), color='y')
ax.scatter(*centre, color='g')
plt.show()

In [ ]:
split_roads_gdf = gpd.GeoDataFrame({"geometry": split_roads, "id": range(len(split_roads))})
graph = momepy.gdf_to_nx(remaining_roads_gdf)

In [ ]:
nodes, edges, sw = momepy.nx_to_gdf(graph, points=True, lines=True,
                                    spatial_weights=True)
nodes
sw

In [ ]:
import networkx as nx 

In [ ]:
graph.nodes

In [ ]:
source = (442745.0, 111065.0)
target = (441391.415, 111891.2990000006)
paths = [p for p in nx.all_shortest_paths(graph, source=source, target=target)]
x, y = zip(*paths[0])

In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
# roads.plot(ax=ax)
remaining_roads_gdf.plot(ax=ax)
# new_lines_gdf.plot(color="r", ax=ax)
ax.scatter(x, y, color='r')
ax.scatter(*centre, color='g')
ax.scatter(*source, color='g')
ax.scatter(*target, color='g')
plt.show()

In [ ]:
import networkx as nx
import momepy

nx.load_centrality()